## Dropout

In [1]:
# coding:utf-8
import torch
import numpy as np
import torch.nn as nn

- 原始论文：http://www.cs.toronto.edu/~rsalakhu/papers/srivastava14a.pdf

考虑一个带有$L$层隐藏层的神经网络，$l \in \{1, \cdots, L\}$。$z^{(l)}$定义为$l$层网络的输入向量，$y^{(l)}$定义为$l$层网络的输出向量，$w^{(l)}$定义为$l$层网络的权重，$b^{(l)}$定义为$l$层网络的偏置。标准的前馈神经网络可以描述为：

其中$f$为激活函数，比如$f(x)=1/(1+exp(-x))$。基于dropout，前馈网络可以表示为：

$$
\begin{aligned}
z_{i}^{(l+1)} &=\mathbf{w}_{i}^{(l+1)} \mathbf{y}^{l}+b_{i}^{(l+1)} \\
y_{i}^{(l+1)} &=f\left(z_{i}^{(l+1)}\right)
\end{aligned}
$$

添加完dropout之后的话，前馈网络可以表示为：

$$
\begin{aligned}
r_{j}^{(l)} & \sim \operatorname{Bernoulli}(p) \\
\widetilde{\mathbf{y}}^{(l)} &=\mathbf{r}^{(l)} * \mathbf{y}^{(l)}, \\
z_{i}^{(l+1)} &=\mathbf{w}_{i}^{(l+1)} \widetilde{\mathbf{y}}^{l}+b_{i}^{(l+1)} \\
y_{i}^{(l+1)} &=f\left(z_{i}^{(l+1)}\right)
\end{aligned}
$$

但是dropout实现由两种方式，一种是上面的`vanilla`版本，但是在做`inference`的时候，数据会进行$1-p$倍的缩小。`inverted dropout`在丢弃完之后，进行$1/(1-p)$倍的放大。反向传播的时候也是同步放大，但是在做`inference`的时候就不需要额外的处理了。

### 1. Numpy实现

In [2]:
def dropout(x, level):
    """
    # dropout函数的实现，函数中，x是本层网络的激活值。Level就是dropout就是每个神经元要被丢弃的概率。
    """
    if level < 0. or level >= 1: #level是概率值，必须在0~1之间
        raise ValueError('Dropout level must be in interval [0, 1[.')
    retain_prob = 1. - level
 
    # 我们通过binomial函数，生成与x一样的维数向量。binomial函数就像抛硬币一样，我们可以把每个神经元当做抛硬币一样
    # 硬币 正面的概率为p，n表示每个神经元试验的次数
    # 因为我们每个神经元只需要抛一次就可以了所以n=1，size参数是我们有多少个硬币。
    #即将生成一个0、1分布的向量，0表示这个神经元被屏蔽，不工作了，也就是dropout了
    random_tensor = np.random.binomial(n=1, p=retain_prob, size=x.shape) 
    print("random_tensor: ", random_tensor)
 
    x *= random_tensor
    print("x: ", x)
    x /= retain_prob
 
    return x

In [3]:
#对dropout的测试，大家可以跑一下上面的函数，了解一个输入x向量，经过dropout的结果  
x=np.asarray([1,2,3,4,5,6,7,8,9,10],dtype=np.float32)
dropout(x,0.4)

random_tensor:  [0 0 1 0 1 0 0 0 1 1]
x:  [ 0.  0.  3.  0.  5.  0.  0.  0.  9. 10.]


array([ 0.      ,  0.      ,  5.      ,  0.      ,  8.333333,  0.      ,
        0.      ,  0.      , 14.999999, 16.666666], dtype=float32)

### 2. Torch实现

参数`p`表示不保留节点的比例。

In [4]:
m = nn.Dropout(p=0.5)
data = torch.randn(4, 5)
data

tensor([[ 0.4935,  1.3364,  0.4463, -0.8671,  0.6874],
        [-0.2558, -0.9374, -0.9058,  0.9340,  0.5590],
        [ 0.0684, -0.2466,  0.6334,  1.1733,  1.7996],
        [-1.4010,  1.0581, -0.4400, -2.4728,  0.8179]])

In [5]:
output = m(data)
output

tensor([[ 0.0000,  0.0000,  0.0000, -0.0000,  0.0000],
        [-0.0000, -1.8748, -0.0000,  1.8680,  0.0000],
        [ 0.0000, -0.0000,  1.2668,  2.3465,  3.5991],
        [-0.0000,  0.0000, -0.0000, -4.9457,  0.0000]])